In [ ]:
!git clone https://github.com/park1200656/KnuSentiLex

In [ ]:
!pip install AST

In [ ]:
!pip install -U pandas-profiling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import ast

!pip3 install pyldavis
import pyLDAvis.sklearn
from tqdm import tqdm
from konlpy.tag import Okt
import json
import matplotlib.pyplot as plt
#import MeCab
import os
import jpype
from matplotlib import rc
okt = Okt()

In [ ]:
# 감성 사전으로 단어를 평가하는 함수
# token = 문자열 단어
def score_word(token):
    with open('KnuSentiLex/data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
        data = json.load(f)
    result = ['None','None']
    for i in range(0, len(data)):
        if data[i]['word'] == token:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])
    return result[1]

In [ ]:
# 문장을 평가하는 함수
# sentence = 문자열 문장
def score_sentence(sentence):
    tokens = okt.pos(sentence, stem = True)
    
    token = []
    test = []
    for i in tokens:
        token.append(i[0])
        test.append(score_word(i[0]))
    
    prenegative = ["안", "못"]
    sunegative = ["않다", "모르다"]
    adjust = []
    
    for a, i in enumerate(test):

        try:
            if token[a - 1] in prenegative:
                adjust.append(int(i) * -1)
            else:
                adjust.append(i)
        except:
            adjust.append(i)
            
    adjust2 = []
    for a, i in enumerate(adjust):        
        try:
            if token[a + 1] in sunegative:
                adjust2.append(int(i) * -1)
            else:
                adjust2.append(i)
        except:
            adjust2.append(i)
    
    while True:
        try:
            adjust2.remove("None")
        except:
            break
    
    test2 = []
    for i in adjust2:
        test2.append(int(i))
        
    try:
        test2[-1] = test2[-1] * 3
    except:
        pass

    try:
        return sum(test2)/len(test2)
    except:
        return 0

# 코드 적용

In [ ]:
data = pd.read_csv("data.csv", index_col = 0)
data

- 각 클러스터별로 모으기

In [ ]:
#클러스터끼리 모으기
clu0 = data[data['cluster'] == 0].reset_index(drop = True)
clu1 = data[data['cluster'] == 1].reset_index(drop = True)
# clu2 = data[data['cluster'] == 2].reset_index(drop = True)
clu3 = data[data['cluster'] == 3].reset_index(drop = True)
# clu4 = data[data['cluster'] == 4].reset_index(drop = True)
clu5 = data[data['cluster'] == 5].reset_index(drop = True)

In [ ]:
#tfidf 단어 중에서 키워드 5개 선정
word_list0 = ['물레길', '자라섬', '숙정문', '비진도', '백악산']
word_list1 = ['펩타이드', '여에스더', '락토바실러스' ,'콜라겐', '콜레올로']
# word_list2 = ['기념일', '카네이션', '결혼기념일', '생신', '스승의날']
word_list3 = ['이퀼리브리엄', '마르티니크', '라치몬트', '메인스트리트' ,'스쿱쿠키']
# word_list4 = ['월세', '대출', '고시텔', '보증금', '전세']
word_list5 = ['참다랑어', '곰치', '비빔국수', '순두부찌개', '곱창전골']

In [ ]:
# 아자아자 화이팅!!
def lets_go(cluster, word_list):
    
    target = []
    target_num = []
    index = []
    for i, text in enumerate(cluster['review']) : #키워드 중 하나라도 포함되어 있는 게시글 수집
        for word in word_list :
            if word in ast.literal_eval(text):
                
                target_num.append(ast.literal_eval(text).index(word))
                target.append(text)
                index.append(i)
                break
    
    importance = len(target)
    print("키워드를 포함하는 게시물 수: ", len(target))
    target = list(target)
    
    sentiment = []
    for i, text in tqdm(enumerate(target)): #포함된 게시물에 한해서만 감성분석 실시
        tokens = ast.literal_eval(text) #review i번째 토큰화
        
        num = 0
        
        try:
            for token in tokens[target_num[i]-5:target_num[i]+5]:
                if score_word(token) == 'None':
                    pass
                else:
                    num += float(score_word(token))
                    
            sentiment.append(num)
                    
        except :
            sentiment.append(0)
                             
        
                             
    cluster['sentiment'] = 0
    
    k = 0
    for dex in index :

        cluster['sentiment'][dex] = sentiment[k]
        k += 1
        
    return cluster, importance

In [ ]:
clu0, clu0_importance = lets_go(clu0, word_list0)
clu0.to_csv('sentiment0.csv', encoding = 'utf-8-sig')

clu1, clu1_importance = lets_go(clu1, word_list1)
clu1.to_csv('sentiment1.csv', encoding = 'utf-8-sig')

# clu2, clu2_importance = lets_go(clu2, word_list2)
# clu2.to_csv('sentiment2.csv', encoding = 'utf-8-sig')

clu3, clu3_importance = lets_go(clu3, word_list3)
clu3.to_csv('sentiment3.csv', encoding = 'utf-8-sig')

# clu4, clu4_importance = lets_go(clu4, word_list4)
# clu4.to_csv('sentiment4.csv', encoding = 'utf-8-sig')

clu5, clu5_importance = lets_go(clu5, word_list5)
clu5.to_csv('sentiment5.csv', encoding = 'utf-8-sig')

In [ ]:
satisfaction0 = clu0.sentiment.sum() / clu0_importance
satisfaction1 = clu1.sentiment.sum() / clu1_importance
# satisfaction2 = clu2.sentiment.sum() / clu2_importance
satisfaction3 = clu3.sentiment.sum() / clu3_importance
# satisfaction4 = clu4.sentiment.sum() / clu4_importance
satisfaction5 = clu5.sentiment.sum() / clu5_importance

In [ ]:
sentiment = [satisfaction0, satisfaction1, satisfaction3, satisfaction5] #클러스터 갯수에 맞춰서 수정

#  만족도 점수를 도출
satisfaction = []
for i in sentiment:
    stm = 10 * ((i - min(sentiment)) / (max(sentiment) - min(sentiment)))
    satisfaction.append(stm)
satisfaction

In [ ]:
size = [clu0_importance, clu1_importance,clu3_importance, clu5_importance]

importance = []
for i in size:
    sz = 10 * ((i - min(size)) / (max(size) - min(size)))
    importance.append(sz)
importance

# 시각화

In [ ]:
def min_max_scaling(series):
    minn = np.min(series)
    maxx = np.max(series)
    return [(i-minn)/(maxx-minn) for i in series]

def visualize(importance, satisfaction):
    jwapyo = []
    ddex = []
    if len(importance) != len(satisfaction):
        raise Exception("Importance와 Satisfaction의 길이가 다릅니다.")

    for i in range(len(importance)):
        jwapyo.append([importance[i], satisfaction[i]])
    
    x, y = zip(*jwapyo)
    colors = ["purple", "r", "b", "darkblue", "navy", 'gold', 'goldenrod', 'darkgoldenrod', 'khaki', 'g', "lime", "forestgreen"]
    for i in range(len(x)):
        plt.scatter(x[i], y[i], label = labels[i], color = colors[i])

    plt.legend(loc = "upper right", bbox_to_anchor=(1.3, 1))

    plt.xlabel("Scale", fontsize = 12)
    plt.ylabel("Sentiment", fontsize = 12)

    plt.show()

In [ ]:
dfs = pd.DataFrame({'Importance': importance, 
                   'Satisfaction': satisfaction})
dfs = dfs.fillna(0)
dfs

In [ ]:
import pickle

with open('fixed_sentiment.txt','wb') as f:
    pickle.dump(dfs, f) #쓰기모드

In [ ]:
import pickle

with open('fixed_sentiment.txt','rb') as f:
    dfs = pickle.load(f) #불러오기, 읽기모드

In [ ]:
dfs